In [35]:
# Day 8 Part 1

def get_accumulator_value(instructions):
    idx = 0
    accumulator = 0
    instructions = [[x[:3], int(x[4:]), False] for x in instructions]
    
    while(True):
        if instructions[idx][2]:
            return accumulator
        
        instructions[idx][2] = True
        if instructions[idx][0] == 'nop':
            idx = idx + 1
        elif instructions[idx][0] == 'acc':
            accumulator = accumulator + instructions[idx][1]
            idx = idx + 1
        elif instructions[idx][0] == 'jmp':
            idx = idx + instructions[idx][1]
    
# Test first
with open('test_data/day8.txt', 'r') as in_file:
    test_data = in_file.read().split('\n')

print("Running on test data. Verifying result...")
print(get_accumulator_value(test_data) == 5)
print('')

# Now run on actual data
with open('data/day8.txt', 'r') as in_file:
    real_data = in_file.read().split('\n')

print("Puzzle answer is: " + str(get_accumulator_value(real_data)))

Running on test data. Verifying result...
True

Puzzle answer is: 1675


In [46]:
# Day 8 Part 2

import copy

def get_accumulator_value2(instructions):
    idx = 0
    accumulator = 0
    
    while(True):                
        if idx == len(instructions):
            return True, accumulator
        if instructions[idx][2]:
            return False, accumulator
        
        instructions[idx][2] = True
        if instructions[idx][0] == 'nop':
            idx = idx + 1
        elif instructions[idx][0] == 'acc':
            accumulator = accumulator + instructions[idx][1]
            idx = idx + 1
        elif instructions[idx][0] == 'jmp':
            idx = idx + instructions[idx][1]

def fix_infinite_loop(instructions):
    instructions = [[x[:3], int(x[4:]), False] for x in instructions]
    for i in range(0, len(instructions)):
        inst_copy = copy.deepcopy(instructions)
        if inst_copy[i][0] == 'nop':
            inst_copy[i][0] = 'jmp'
        elif inst_copy[i][0] == 'jmp':
            inst_copy[i][0] = 'nop'
        else:
            continue
        
        success, acc = get_accumulator_value2(inst_copy)
        if success:
            return success, acc
    return False, -1
            
    
print("Running on test data. Verifying result...")
print(fix_infinite_loop(test_data) == (True, 8))
print('')

# Now run on actual data
print("Puzzle answer is: " + str(fix_infinite_loop(real_data)[1]))

Running on test data. Verifying result...
True

Puzzle answer is: 1532


In [63]:
# Day 9 Part 1

import numpy as np

def can_find_two_entries(data, target_number):
    for item in data:
        temp = target_number - item
        try:
            data.tolist().index(temp)
            return True
        except:
            pass
    return False

def find_weakness(data, preamble_size):
    for i in range(preamble_size, len(data)):
        preamble = np.sort(data[i-preamble_size:i])
        number = data[i]
        if not can_find_two_entries(preamble, number):
            return number

test_data = np.loadtxt('test_data/day9.txt', dtype=np.int64)
print("Running on test data. Verifying result...")
print(find_weakness(test_data, 5) == 127)
print('')

real_data = np.loadtxt('data/day9.txt', dtype=np.int64)
print("Puzzle answer is: " + str(find_weakness(real_data, 25)))

Running on test data. Verifying result...
True

Puzzle answer is: 1309761972


In [68]:
# Day 9 Part 2

def find_contiguous_sum(data, list_width, target_number):
    for i in range(0, len(data) - list_width):
        if np.sum(data[i:i + list_width]) == target_number:
            return data[i:i + list_width]
    
def crack_encryption(data, target_number):
    for list_width in range(2, len(data)-1):
        numbers = find_contiguous_sum(data, list_width, target_number)
        if numbers is not None:
            numbers = np.sort(numbers)
            return numbers[0] + numbers[list_width-1]
    
test_data = np.loadtxt('test_data/day9.txt', dtype=np.int64)
print("Running on test data. Verifying result...")
print(crack_encryption(test_data, 127) == 62)
print('')

real_data = np.loadtxt('data/day9.txt', dtype=np.int64)
print("Puzzle answer is: " + str(crack_encryption(real_data, 1309761972)))

Running on test data. Verifying result...
True

Puzzle answer is: 177989832


In [115]:
# Day 10 Part 1

def count_1_and_3_jolt_differences(data):
    data = np.sort(data)
    diff1 = 0
    diff3 = 1  #always will have a diff3 at the end
    if data[0] == 1:
        diff1 = 1
    if data[0] == 3:
        diff3 = diff3 + 1
    
    for i in range(1, len(data)):
        if data[i] - data[i-1] == 1:
            diff1 = diff1 + 1
        if data[i] - data[i-1] == 3:
            diff3 = diff3 + 1
    return diff1, diff3
            
test_data = np.loadtxt('test_data/day10.txt', dtype=np.int32)
test_data2 = np.loadtxt('test_data/day10_larger.txt', dtype=np.int32)
print("Running on test data. Verifying result...")
print(count_1_and_3_jolt_differences(test_data) == (7, 5))
print(count_1_and_3_jolt_differences(test_data2) == (22, 10))
print('')

real_data = np.loadtxt('data/day10.txt', dtype=np.int32)
diff1, diff3 = count_1_and_3_jolt_differences(real_data)
print("Puzzle answer is: " + str(diff1*diff3))

Running on test data. Verifying result...
True
True

Puzzle answer is: 2590


In [131]:
# Day 10 Part 2
count_cache = {}

def prepare_data(data):
    sorted_data = np.sort(data)
    temp = [0]
    temp.extend([x for x in sorted_data])
    temp.append(sorted_data[-1] + 3)
    return temp

def dynamic_count_combinations(data, i):
    if i == len(data) - 1:
        return 1
    
    if i in count_cache:
        return count_cache[i]
    
    count = 0
    for j in range(i + 1, len(data)):
        if data[j] - data[i] <= 3:
            count += dynamic_count_combinations(data, j)

    count_cache[i] = count
    return count

def count_combinations(data):
    count_cache.clear()
    return dynamic_count_combinations(prepare_data(data), 0)

print("Running on test data. Verifying result...")
print(count_combinations(test_data) == 8)
print(count_combinations(test_data2) == 19208)
print('')

print("Puzzle answer is: " + str(count_combinations(real_data)))

Running on test data. Verifying result...
True
True

Puzzle answer is: 226775649501184


In [206]:
# Day 11 Part 1
import copy

def read_layout(file_path):
    with open(file_path, 'r') as in_file:
         return [list(x) for x in in_file.read().split('\n')]
    
def print_layout(layout):
    for i in range(0, len(layout)):
        print(layout[i])

def count_differences(layout1, layout2):
    count = 0
    for i in range(0, len(layout1)):
        for j in range(0, len(layout1[0])):
            if layout1[i][j] != layout2[i][j]:
                count = count + 1
    return count
    
def count_ajacent_seats(data, x, y, state):
    count = 0
    for i in range( max(0, x-1), min(x+2, len(data))):
        for j in range(max(0, y-1), min(y+2, len(data[0]))):
            if i == x and j == y:
                continue
            if data[i][j] == state:
                count = count + 1
    return count

def iterate_seat_layout(data, iterations):
    if iterations == 0:
        return data
    
    width = len(data)
    length = len(data[0])
    
    new_layout = copy.deepcopy(data)
    for x in range(0, width):
        for y in range(0, length):
            if data[x][y] == 'L' and count_ajacent_seats(data, x, y, '#') == 0:
                new_layout[x][y] = '#'
            elif data[x][y] == '#' and count_ajacent_seats(data, x, y, '#') >= 4:
                new_layout[x][y] = 'L'
    
    if new_layout == data:
        print('Converged!')
        return new_layout
    else:
        return iterate_seat_layout(new_layout, iterations-1)


test_data = read_layout('test_data/day11_start.txt')
test_data_1 = read_layout('test_data/day11_p1_1.txt')
test_data_2 = read_layout('test_data/day11_p1_2.txt')
test_data_3 = read_layout('test_data/day11_p1_3.txt')
test_data_4 = read_layout('test_data/day11_p1_4.txt')
test_data_5 = read_layout('test_data/day11_p1_end.txt')


print("Running on test data. Verifying result...")
print(iterate_seat_layout(test_data, 1) == test_data_1)
print(iterate_seat_layout(test_data, 2) == test_data_2)
print(iterate_seat_layout(test_data, 3) == test_data_3)
print(iterate_seat_layout(test_data, 4) == test_data_4)
print(iterate_seat_layout(test_data, 5) == test_data_5)
print('Count differences')
print(count_differences(test_data, iterate_seat_layout(test_data, 5)) == 37)

real_data = read_layout('data/day11.txt')
print(count_differences(real_data, iterate_seat_layout(real_data, 100)))

Running on test data. Verifying result...
True


In [222]:
# Day 11 Part 2
def count_visible_seats(data, x, y):
    count = 0
    for angle in [(0,1), (0,-1), (1,0), (-1,0), (1,1), (1,-1), (-1,-1), (-1, 1)]:    
        i = x
        j = y
        while i + angle[0] >= 0 and i + angle[0] < len(data) and j + angle[1] >=0 and j + angle[1] < len(data[0]):
            i = i + angle[0]
            j = j + angle[1]
            if data[i][j] == 'L':
                break
            if data[i][j] == '#':
                count = count + 1
                break
    return count

def iterate_seat_layout2(data, iterations):
    if iterations == 0:
        return data
    
    width = len(data)
    length = len(data[0])
    
    new_layout = copy.deepcopy(data)
    for x in range(0, width):
        for y in range(0, length):
            if data[x][y] == 'L' and count_visible_seats(data, x, y) == 0:
                new_layout[x][y] = '#'
            elif data[x][y] == '#' and count_visible_seats(data, x, y) >= 5:
                new_layout[x][y] = 'L'
    
    if new_layout == data:
        print('Converged!')
        return new_layout
    else:
        return iterate_seat_layout2(new_layout, iterations-1)

    
test_data = read_layout('test_data/day11_start.txt')
test_data_1 = read_layout('test_data/day11_p2_1.txt')
test_data_2 = read_layout('test_data/day11_p2_2.txt')
test_data_3 = read_layout('test_data/day11_p2_3.txt')
test_data_4 = read_layout('test_data/day11_p2_4.txt')
test_data_5 = read_layout('test_data/day11_p2_5.txt')
test_data_6 = read_layout('test_data/day11_p2_end.txt')

print("Running on test data. Verifying result...")
print(iterate_seat_layout2(test_data, 1) == test_data_1)
print(iterate_seat_layout2(test_data, 2) == test_data_2)
print(iterate_seat_layout2(test_data, 3) == test_data_3)
print(iterate_seat_layout2(test_data, 4) == test_data_4)
print(iterate_seat_layout2(test_data, 5) == test_data_5)
print(iterate_seat_layout2(test_data, 6) == test_data_6)
print('Count differences')
print(count_differences(test_data, iterate_seat_layout2(test_data, 6)) == 26)

real_data = read_layout('data/day11.txt')
print(count_differences(real_data, iterate_seat_layout2(real_data, 100)))

Running on test data. Verifying result...
True
True
True
True
True
True
Count differences
True
Converged!
1937


In [15]:
# Day 12 Part 1

class Ship:
    def __init__(self):
        self.directions = {0 : 'E', 90: 'S', 180: 'W', 270: 'N'}
        self.facing = 0
        self.north_south = 0
        self.east_west = 0
        
    def get_manhattan_distance(self):
        return abs(self.north_south) + abs(self.east_west)

    def navigate(self, direction, amount):
        if direction == 'N':
            self.north_south = self.north_south + amount
        elif direction == 'S':
            self.north_south = self.north_south - amount
        elif direction == 'E':
            self.east_west = self.east_west + amount
        elif direction == 'W':
            self.east_west = self.east_west - amount
        elif direction == 'F':
            self.navigate(self.directions[self.facing], amount)
        elif direction == 'R':
            self.facing = (self.facing + amount) % 360
        elif direction == 'L':
            self.facing = (self.facing - amount) % 360
        else:
            pass
    
def navigate_ship(instructions):
    ship = Ship()
    for instruction in instructions:
        ship.navigate(instruction[:1], int(instruction[1:]))
    return ship.get_manhattan_distance()
    
# Test first
with open('test_data/day12.txt', 'r') as in_file:
    test_data = in_file.read().split('\n')

print("Running on test data. Verifying result...")
print(navigate_ship(test_data) == 25)
print('')

# Now run on actual data
with open('data/day12.txt', 'r') as in_file:
    real_data = in_file.read().split('\n')

print("Puzzle answer is: " + str(navigate_ship(real_data)))

Running on test data. Verifying result...
True

Puzzle answer is: 1152


In [47]:
# Day 12 Part 2
import math

class Ship2:
    def __init__(self):
        self.waypoint_pos = [-10, 1]
        self.pos = np.array([0, 0]
        
    def get_manhattan_distance(self):
        return abs(self.pos[0]) + abs(self.pos[1])
    
    def rotate_waypoint(self, degrees):
        radians = math.radians(degrees)
        x = round(self.waypoint_pos[0] * math.cos(radians) + self.waypoint_pos[1] * math.sin(radians))
        y = round(-self.waypoint_pos[0] * math.sin(radians) + self.waypoint_pos[1] * math.cos(radians))
        self.waypoint_pos = [x, y]

    def navigate(self, direction, amount):
        if direction == 'N':
            self.waypoint_pos[1] = self.waypoint_pos[1] + amount
        elif direction == 'S':
            self.waypoint_pos[1] = self.waypoint_pos[1] - amount
        elif direction == 'E':
            self.waypoint_pos[0] = self.waypoint_pos[0] - amount
        elif direction == 'W':
            self.waypoint_pos[0] = self.waypoint_pos[0] + amount
        elif direction == 'F':
            self.pos[0] = self.pos[0] + amount * self.waypoint_pos[0]
            self.pos[1] = self.pos[1] + amount * self.waypoint_pos[1]
        elif direction == 'L':
            self.rotate_waypoint(amount)
        elif direction == 'R':
            self.rotate_waypoint(-amount % 360)
        else:
            pass

def navigate_ship2(instructions):
    ship = Ship2()
    for instruction in instructions:
        ship.navigate(instruction[:1], int(instruction[1:]))
    return ship.get_manhattan_distance()

print("Running on test data. Verifying result...")
print(navigate_ship2(test_data) == 286)
print('')

print("Puzzle answer is: " + str(navigate_ship2(real_data)))

Running on test data. Verifying result...
True

Puzzle answer is: 58637


In [63]:
# Day 13 Part 1

def read_input_timetable(file_path):
    with open(file_path, 'r') as in_file:
        temp = in_file.read().split('\n')
    return int(temp[0]), [int(x) for x in filter(lambda x: x != 'x', temp[1].split(','))]
                     
def get_nearest_bus(timetable, starting_point):
    found = False
    wait_time = 0
    while(True):
        for bus in timetable:
            if (starting_point + wait_time) % bus == 0:
                return bus, wait_time, bus*wait_time
        
        wait_time = wait_time + 1

test_data = read_input_timetable('test_data/day13.txt')
print("Running on test data. Verifying result...")
print(get_nearest_bus(test_data[1], test_data[0]) == (59, 5, 295))
print('')

real_data = read_input_timetable('data/day13.txt')
print("Puzzle answer is: " + str(get_nearest_bus(real_data[1], real_data[0])[2]))

Running on test data. Verifying result...
True

Puzzle answer is: 2305


In [90]:
# Day 13 Part 2
# Brute force fail...

def read_input_timetable2(file_path):
    with open(file_path, 'r') as in_file:
        temp = in_file.read().split('\n')[1].split(',')
    return list(map(lambda x: x if x == 'x' else int(x), temp))

def check_element(bus, timestamp, offset):
    if bus == 'x':
        return True
    else:
        if (timestamp + offset) % bus == 0:
            return True
        return False
    
def find_first_timestamp(timetable):
    timestamp = 0
    while(True):
        if all([check_element(x, timestamp, i) for i, x in enumerate(timetable)]):
            return timestamp
        timestamp = timestamp + 1
    
test_data = read_input_timetable2('test_data/day13.txt')
print("Running on test data. Verifying result...")
print(find_first_timestamp(test_data) == 1068781)
print(find_first_timestamp([17,'x',13,19]) == 3417)
print(find_first_timestamp([67,7,59,61]) == 754018)
print(find_first_timestamp([67,'x',7,59,61]) == 779210)
print(find_first_timestamp([67,7,'x',59,61]) == 1261476)
#print(find_first_timestamp([1789,37,47,1889]) == 1202161486)
print('DONE')

#real_data = read_input_timetable2('data/day13.txt')
#print("Puzzle answer is: " + str(find_first_timestamp(real_data)))

Running on test data. Verifying result...
True
True
True
True
True
DONE


In [118]:
# Day 14 Part 1
import re

inst_p = re.compile('mem\[([0-9]+)\] = ([0-9]+)')

def apply_mask(mask, value):
    temp = "{:b}".format(value)
    temp = list((36 - len(temp)) * '0' + temp)
    
    for i in range(0, 36):
        if mask[i] != 'X':
            temp[i] = mask[i]
    return int("".join(temp), 2)

def run_program(instructions):
    memory = {}
    mask = 36 * 'X'
    for inst in instructions:
        if inst[:4] == 'mask':
            mask = inst[7:]
        else:
            adress, value = inst_p.findall(inst)[0]
            value = apply_mask(mask, int(value))
            memory[adress] = value
    
    final_sum = 0
    for _, value in memory.items():
        final_sum = final_sum + value
    return final_sum


# Test first
with open('test_data/day14.txt', 'r') as in_file:
    test_data = in_file.read().split('\n')
run_program(test_data)
print("Running on test data. Verifying result...")
print(run_program(test_data) == 165)

with open('data/day14.txt', 'r') as in_file:
    real_data = in_file.read().split('\n')
print("Puzzle answer is: " + str(run_program(real_data)))

Running on test data. Verifying result...
True
Puzzle answer is: 17481577045893


In [12]:
# Day 14 Part 2
from math import comb
import re
import itertools

inst_p = re.compile('mem\[([0-9]+)\] = ([0-9]+)')

def get_masked_adresses(mask, adress):
    temp = "{:b}".format(adress)
    temp = list((36 - len(temp)) * '0' + temp)
        
    for i in range(0, 36):
        if mask[i] != '0':
            temp[i] = mask[i]
            
    indices = [i for i, x in enumerate(temp) if x == 'X']
    n_adresses = comb(len(indices), 2)
    all_adresses = []
    #print("".join(temp), indices, n_adresses)
    for combination in [list(i) for i in itertools.product([0, 1], repeat=len(indices))]:
        for i in range(0, len(indices)):
            temp[indices[i]] = str(combination[i])
        all_adresses.append(''.join(temp))
    return all_adresses
    

def run_program(instructions):
    memory = {}
    mask = 36 * 'X'
    for inst in instructions:
        if inst[:4] == 'mask':
            mask = inst[7:]
        else:
            temp, value = inst_p.findall(inst)[0]
            adresses = get_masked_adresses(mask, int(temp))
            for adress in adresses:
                memory[adress] = int(value)
    
    final_sum = 0
    for _, value in memory.items():
        final_sum = final_sum + value
    return final_sum


# Test first
with open('test_data/day14_2.txt', 'r') as in_file:
    test_data = in_file.read().split('\n')
run_program(test_data)
print("Running on test data. Verifying result...")
print(run_program(test_data) == 208)

with open('data/day14.txt', 'r') as in_file:
    real_data = in_file.read().split('\n')
print("Puzzle answer is: " + str(run_program(real_data)))


Running on test data. Verifying result...
True
Puzzle answer is: 4160009892257
